![title](attachment:./title.png)

## Завдання

1. Взяти файл з даними у вiдповiдностi з варiантом.

2. Побудувати частi набори та асоцiативнi правила (АП), використовуючи алгоритм Apriori. Дослiдити множини АП залежно вiд параметрiв побудови АП:

* мiнiмальної пiдтримки,
* мiнiмальної достовiрностi,
* максимальної потужностi частих наборiв.


Пiдiбрати значення параметрiв алгоритму Apriori, якi призводять до
значущих АП. АП Rk вважати значущим, якщо покращення (improvement
або лiфт (lift)): Improv(Rk) > 1.


3. Побудувати частi набори та множину АП, використовуючи алгоритм FPросту.
Дослiдити множини АП залежно вiд параметрiв побудови АП, якi
було наведено вище для Apriori.

Пiдiбрати значення параметрiв алгоритму FP-росту, якi призводять до
значущих АП.

4. Зробити висновки щодо впливу параметрiв алгоритмiв Apriori та FP-росту на знайденi множини АП.
5. Знайти значення прогнозу на основi побудованої множини правил.
6. Порiвняти результати, отриманi алгоритмами Apriori та FP-росту.

#### 6 варіант

Var_6_1000_Transactions

### 1. Взяти файл з даними у вiдповiдностi з варiантом. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder

In [3]:
data = pd.read_csv('Var_6_1000_Transactions.csv')
data

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035
...,...,...
9715,1000,35951
9716,1000,3025
9717,1000,35734
9718,1000,26800


In [2]:
#pip install mlxtend

     |████████████████████████████████| 1.3 MB 27.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [0]:
def make_orders(id_order, id_item):
    result = []
    add = []
    
    curr_id = id_order[0]
    for i in range(len(id_order)):
        if id_order[i] == curr_id:
            add.append(id_item[i])
        else:
            result.append(add)
            add = []
            curr_id = id_order[i]
            add.append(id_item[i])
    result.append(add)
    return result

In [0]:
orders = make_orders(list(data['order_id']), list(data['product_id']))
print('Number of successful orders: ', len(set(data['order_id'])))

print('Orders (10):\n')
for i in range(10):
    print(orders[i])

In [0]:
def encoding(orders):
    encoder = TransactionEncoder()
    data = encoder.fit(orders).transform(orders)
    return pd.DataFrame(data, columns=encoder.columns_)

In [0]:
data_frame = encoding(orders)
data_frame.astype("int")[600:650]

### 2. Побудувати частi набори та асоцiативнi правила (АП), використовуючи алгоритм Apriori.
### +3. Побудувати частi набори та множину АП, використовуючи алгоритм FP-росту.

Дослiдити множини АП залежно вiд параметрiв побудови АП:

* мiнiмальної пiдтримки,
* мiнiмальної достовiрностi,
* максимальної потужностi частих наборiв.

Пiдiбрати значення параметрiв алгоритму Apriori, якi призводять до
значущих АП.

In [0]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [0]:
def rules_support_plot(num_rules, metric_name, metric_values, confs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
    
    ax1.plot(metric_values, num_rules)
    ax1.set_xlabel(metric_name)
    ax1.set_ylabel('number of rules')
    
    ax2.plot(metric_values, confs)
    ax2.set_xlabel(metric_name)
    ax2.set_ylabel('max confidence')
    
    plt.show()

In [0]:
def analyze_support_lenght(df, algorithm):
    max_confs1 = []
    n_rules1 = []
    supports = [0.003+i*10**(-3) for i in range(6)]
    length = [2, 3, 4]
    
    for support in supports:
        print('min_support: ', support)
        
        rules = association_rules(algorithm(df, min_support=support), metric='confidence', min_threshold=0.5)
        display(rules)
        
        n_rules1.append(len(rules.index))
        max_confs1.append(max(list(rules['confidence'])))
        
        n_rules2 = []
        max_confs2 = []
        
        for lng in length:
            
            rules = association_rules(algorithm(df, min_support=support, max_len = lng), metric='confidence', min_threshold=0.5)
            n_rules2.append(len(rules.index))
            if(len(list(rules['confidence']))):
                max_confs2.append(max(list(rules['confidence'])))
            else:
                max_confs2.append(0.0)
        
        rules_support_plot(n_rules2, 'max_len', length, max_confs2)
          
    rules_support_plot(n_rules1,'min_support', supports, max_confs1)
        
        

In [0]:
def rules_confidence_plot(num_rules, confidence):
    plt.plot(confidence, num_rules)
    plt.xlabel('min_confidence')
    plt.ylabel('number of rules')
    plt.show()

In [0]:
def analyze_confidence(df, algorithm):
    confs = [i*0.1 for i in range(5, 11)]
    n_rules = []
    
    for conf in confs:
        print('Confidence: ', conf)
        rules = association_rules(algorithm(df, min_support=0.004, max_len = 3), metric='confidence', min_threshold=conf)
        display(rules)
        n_rules.append(len(rules.index))
        
    rules_confidence_plot(n_rules, confs)

##### Apriory

In [0]:
analyze_support_lenght(data_frame, apriori)

In [0]:
analyze_confidence(data_frame, apriori)

##### FP-growth

In [0]:
analyze_support_lenght(data_frame, fpgrowth)

In [0]:
analyze_confidence(data_frame, fpgrowth)

### 4. Зробити висновки щодо впливу параметрiв алгоритмiв Apriori та FP-росту на знайденi множини АП.

Оскільки максимально можлива підтримка є досить малою, то всі дослідження параметрів проводилися для достовірності >0.5

1. З графаків залежності кількості прикладів від максимальної потужності частих наборів видно, що зі збільшенням мінімальної підтримки зменшується максимальна потужність.

2. З графіків залежності кількості прикладів від мінімальної підтримки видно, що до значення support = 0.004 кількість прикладів різко падає, а починаючи від 0.004 спадає дуже повільно.

Ці два пункти є досить інтуїтивно зрозумілі з самого означення підтримки та достовірності. Очевидно, що набори з великою потужністю будуть зустрічатися рідше, ніж малою потужністю. До того ж кількість наборів, які зустрічаються рідше є більшою, ніж наборів, які зустрічаються часто; звідси стрімкий спад кількості прикладів до пітримки при дуже малій підтриці, адже кожен набір зустрічається хоча б 1 раз (support = 0.001, цей варіант навіть не розлядався).

3. Зі збільшенням достовірності зменшується кількість правил.
4. Зі збільшенням  підтримки зменшується достовірність правил.

### 5. Знайти значення прогнозу на основi побудованої множини правил.

In [0]:
def make_predict(df, algorithm):
    rules = association_rules(algorithm(df, min_support=0.002), metric='lift', min_threshold=1.0)
    r=rules[rules['confidence']==1].sort_values('support', ascending=[False]).head()
    display(r)
    r=rules[rules['support'] > 0.01].sort_values('support', ascending=[False]).head()
    display(r)
    r=rules[rules['lift'] > 100].sort_values('lift', ascending=[False]).head(20)
    display(r)
    r=rules[(rules['support']>0.005)&(rules['confidence']>0.7)]
    display(r)

##### Apriori

In [1]:
make_predict(data_frame, apriori)

NameError: NameError: name 'make_predict' is not defined

##### FP-growth

In [0]:
make_predict(data_frame, fpgrowth)



1. Як можна побачити з таблиць, багато правил має достовірність, рівну 1, що означає, що товар-"antecedents", купується лише з товаром-"consequents". Однак підтримка цих правил є дуже малою. Максимального значення пітримки (0.004184) при цьому набуває 5 правил, проте найбільше значення метрики 'lift' має правило:

(4173) -> (2975)

2. Оскільки обсяг пропозиції товарів (>49683, з них 4438 куплено хоча б 1 раз) значно перевищує кількість покупок (956 успішних замовлень, в сумі 9720 куплених товарів), при чому кількість товарів в одному замовленні є невеликою, то максимальна можлива підтримка є досить малою. Максимального значення (0.025105) набуває 2 правила, але вищу достовірність має правило:

(1953) -> (2221)

3. Максимального значення метрики 'lift' набуває багато правил, які при цьому мають максимальну достовірність, проте дуже малу підтримку (зустрічаються ~2 рази з 956).


* Правило, яке має не максимальні, але відносно високі метрики:

(2313, 1953) -> (2221)


Зважаючи на велику різницю між пропозицією та замовленями, вибрати 1 найточніше правило не можна, бо якщо правила зустрічаються частіше, то мають малу достовірність, а якщо мають максимальну достовірність, то зустрічаються <5 раз.

### 6. Порiвняти результати, отриманi алгоритмами Apriori та FP-росту.

З побудованих в ході роботи таблиць та графіків видно, що обидва алгоритми повертають одинаковий результат. Оскільки двома алгоритмами опрацьовувалися одинакові початкові дані, це свідчить про коректність роботи програми.